<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_4%EC%9D%BC%EC%B0%A8_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [2]:
!pip install pyspark==3.0.1 py4j==0.10.9 

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Titanic Binary Classification example") \
    .getOrCreate()

Exception: Java gateway process exited before sending its port number

# 타이타닉 생존 예측 모델 만들기




In [ ]:
spark

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv

In [ ]:
data = spark.read.csv('./titanic.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

In [ ]:
data.show()

In [ ]:
data.select(['*']).describe().show()

**데이터 클린업**: 

*   PassengerID, Name, Ticket, Embarked는 사용하지 않을 예정 (아무 의미가 없음).
*   Cabin도 비어있는 값이 너무 많아서 사용하지 않을 예정
*   Age는 중요한 정보인데 비어있는 레코드들이 많아서 디폴트값을 채워줄 예정
*   Gender의 경우 카테고리 정보이기에 숫자로 인코딩 필요



In [ ]:
final_data = data.select(['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare'])

In [ ]:
final_data.show()

Age는 평균값으로 채운다

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
imputer_model = imputer.fit(final_data)
final_data = imputer_model.transform(final_data)

In [ ]:
final_data.select("Age", "AgeImputed").show()

성별 정보 인코딩: male -> 0, female -> 1

In [ ]:
from pyspark.ml.feature import StringIndexer

gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')
gender_indexer_model = gender_indexer.fit(final_data)
final_data = gender_indexer_model.transform(final_data)

In [ ]:
final_data.select("Gender", "GenderIndexed").show()

## 피쳐 벡터를 만들기

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeImputed', 'GenderIndexed'], outputCol='features')
data_vec = assembler.transform(final_data)

In [ ]:
data_vec.show()

## 훈련용과 테스트용 데이터를 나누고 binary classification 모델을 하나 만든다

In [ ]:
train, test = data_vec.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

algo = LogisticRegression(featuresCol="features", labelCol="Survived")
model = algo.fit(train)

## 모델 성능 측정

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.select(['Survived','prediction', 'probability']).show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')
evaluator.evaluate(predictions)